In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install tf_keras

In [ ]:
data = pd.read_csv('/kaggle/input/14features-010-jak/features_010_jakteristics_14.csv')

In [ ]:
data.columns

In [ ]:
data = data.drop(['Unnamed: 0'], axis=1)

In [ ]:
data = data.dropna()

In [ ]:
data = data.reset_index()
data

In [ ]:
data = data.astype(float)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
columns_to_scale = ['z', 'eigenvalue_sum', 'omnivariance', 'eigenentropy',
       'anisotropy', 'planarity', 'linearity', 'PCA1', 'PCA2',
       'surface_variation', 'sphericity', 'verticality', 'nx', 'ny', 'nz']

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data[columns_to_scale])
scaled_df = pd.DataFrame(scaled_data, columns=columns_to_scale)
data[columns_to_scale] = scaled_df
data

In [ ]:
# x = data[['Column1','Column2','Column3','Column4','Column5','Column6','Column7','Column8']]
X = data[['z', 'eigenvalue_sum', 'omnivariance', 'eigenentropy',
       'anisotropy', 'planarity', 'linearity', 'PCA1', 'PCA2',
       'surface_variation', 'sphericity', 'verticality', 'nx', 'ny', 'nz']]

y = data[['label']]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import tf_keras

num_classes = 4
classes_present = [1, 2, 5, 8]
class_mapping = {cls: i for i, cls in enumerate(classes_present)}
y_mapped = y_train['label'].map(class_mapping)
y_onehot = tf.one_hot(y_mapped, depth=num_classes)


def create_bnn(input_shape, output_shape):
    model = tf_keras.Sequential([
        tfp.layers.DenseFlipout(64, activation='relu', input_shape=input_shape),
        tfp.layers.DenseFlipout(64, activation='relu'),
        tfp.layers.DenseFlipout(output_shape, activation='softmax')
    ])
    return model

def neg_log_likelihood(y_true, y_pred):
    return -tf.reduce_mean(y_pred.log_prob(y_true))

num_features = 15
input_shape = (num_features,)
output_shape = 4
model = create_bnn(input_shape, output_shape)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_onehot, epochs=10, batch_size=32, validation_split=0.3)

In [ ]:
#for calculation of epistemic uncertainty
# predictions = [model.predict(X_test) for _ in range(100)]
# predictions = tf.stack(predictions)

# # Calculate mean and variance of predictions
# pred_mean = tf.reduce_mean(predictions, axis=0)
# pred_variance = tf.math.reduce_variance(predictions, axis=0)

# # Convert to numpy arrays for further use
# pred_mean = pred_mean.numpy()
# pred_variance = pred_variance.numpy()

# # Example: Print predictions and uncertainties for the first test sample
# print("Predicted class probabilities:", pred_mean[0])
# print("Prediction uncertainty (variance):", pred_variance[0])